# Sequence 2 Sequence
#### Reference:
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

In [25]:
from torch import optim

import codecs
import random
import re
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

# activate cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

TSV_PATH = "C:\\Users\\User\\Desktop\\Ricardo\\KnowledgeGraph_materials\\data_kg\\sequence2sequence\\Sentence pairs in English-Mandarin Chinese - 2021-06-27.tsv"
MAX_LENGTH = 100
MODEL_SAVING_DIR = "C:\\Users\\User\\Desktop\\Ricardo\\KnowledgeGraph_materials\\models_kg\\seq2seq\\"
RESULT_SAVING_DIR = "C:\\Users\\User\\Desktop\\Ricardo\\KnowledgeGraph_materials\\results_kg\\seq2seq\\"

cuda


# Preparing Data

In [2]:
# parsing data

file = codecs.open(TSV_PATH, mode="r", errors="ignore", encoding="utf8")

line_pairs = []

def normalizeString(s):
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

for line in file.readlines():
    line_split_list = line.split("\t")
    line_pairs.append([normalizeString(line_split_list[1].replace("\r\n", "").replace("\n", "")),
                       line_split_list[3].replace("\r\n", "").replace("\n", "")])
    
    
print(line_pairs[:10])

[['Let s try something .', '我們試試看！'], ['I have to go to sleep .', '我该去睡觉了。'], ['Today is June th and it is Muiriel s birthday !', '今天是６月１８号，也是Muiriel的生日！'], ['Muiriel is now .', 'Muiriel现在20岁了。'], ['The password is Muiriel .', '密码是"Muiriel"。'], ['I will be back soon .', '我很快就會回來。'], ['I m at a loss for words .', '我不知道應該說什麼才好。'], ['This is never going to end .', '這個永遠完不了了。'], ['This is never going to end .', '这将永远继续下去。'], ['I just don t know what to say .', '我只是不知道應該說什麼而已……']]


In [3]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name, LNG):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence, LNG):
        if LNG == "EN":
            for word in sentence.split(' '):
                self.addWord(word)
        elif LNG == "ZH":
            for word in sentence:
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def readLangs(lang1, lang2, line_pairs, reverse=False):
    print("Reading lines...")

    # Split every line into pairs and normalize
    pairs = [[line_element for line_element in l] for l in line_pairs]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2, "ZH")
        output_lang = Lang(lang1, "EN")
    else:
        input_lang = Lang(lang1, "EN")
        output_lang = Lang(lang2, "ZH")
        
    print(pairs[0:10])

    return input_lang, output_lang, pairs

In [5]:
def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [6]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, line_pairs, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0], "EN")
        output_lang.addSentence(pair[1], "ZH")
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('EN', 'ZH', False)
print(random.choice(pairs))

Reading lines...
[['Let s try something .', '我們試試看！'], ['I have to go to sleep .', '我该去睡觉了。'], ['Today is June th and it is Muiriel s birthday !', '今天是６月１８号，也是Muiriel的生日！'], ['Muiriel is now .', 'Muiriel现在20岁了。'], ['The password is Muiriel .', '密码是"Muiriel"。'], ['I will be back soon .', '我很快就會回來。'], ['I m at a loss for words .', '我不知道應該說什麼才好。'], ['This is never going to end .', '這個永遠完不了了。'], ['This is never going to end .', '这将永远继续下去。'], ['I just don t know what to say .', '我只是不知道應該說什麼而已……']]
Read 56574 sentence pairs
Trimmed to 56571 sentence pairs
Counting words...
Counted words:
EN 14702
ZH 4547
['I own a horse .', '我拥有一匹马。']


# Model Stage
#### Documetations
* Pytorch Embedding: https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html
* Pytorh GRU: https://pytorch.org/docs/stable/generated/torch.nn.GRU.html
* Pytorch Softmax: https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html
* Pytorch View: https://pytorch.org/docs/stable/generated/torch.Tensor.view.html
* Pytorch Linear: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html

In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [8]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [9]:
def indexesFromSentence(lang, sentence, LNG):
    if LNG == "EN":
        return [lang.word2index[word] for word in sentence.split(' ')]
    elif LNG == "ZH":
        return [lang.word2index[word] for word in sentence]


def tensorFromSentence(lang, sentence, LNG):
    indexes = indexesFromSentence(lang, sentence, LNG)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair, LNG_pair):
    input_tensor = tensorFromSentence(input_lang, pair[0], LNG_pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1], LNG_pair[1])
    return (input_tensor, target_tensor)

In [10]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [11]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [12]:
def trainIters(encoder, decoder, n_iters, LNG_pair, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs), LNG_pair)
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [13]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [14]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 250000, ["EN", "ZH"], print_every=5000)

2m 6s (- 103m 39s) (5000 2%) 4.6258
3m 51s (- 92m 24s) (10000 4%) 4.3789
5m 38s (- 88m 20s) (15000 6%) 4.2040
7m 26s (- 85m 30s) (20000 8%) 4.0454
9m 14s (- 83m 10s) (25000 10%) 3.9528
11m 2s (- 80m 55s) (30000 12%) 3.8119
12m 50s (- 78m 52s) (35000 14%) 3.7262
14m 38s (- 76m 53s) (40000 16%) 3.6921
16m 28s (- 75m 3s) (45000 18%) 3.6569
18m 18s (- 73m 12s) (50000 20%) 3.5803
20m 7s (- 71m 19s) (55000 22%) 3.5139
21m 56s (- 69m 29s) (60000 24%) 3.5140
23m 45s (- 67m 37s) (65000 26%) 3.4625
25m 35s (- 65m 47s) (70000 28%) 3.4554
27m 24s (- 63m 57s) (75000 30%) 3.4252
29m 14s (- 62m 9s) (80000 32%) 3.3885
31m 3s (- 60m 17s) (85000 34%) 3.3592
32m 53s (- 58m 28s) (90000 36%) 3.3893
34m 44s (- 56m 40s) (95000 38%) 3.3465
36m 33s (- 54m 50s) (100000 40%) 3.3350
38m 24s (- 53m 2s) (105000 42%) 3.3161
40m 13s (- 51m 12s) (110000 44%) 3.2806
42m 3s (- 49m 21s) (115000 46%) 3.2676
43m 53s (- 47m 32s) (120000 48%) 3.2789
45m 44s (- 45m 44s) (125000 50%) 3.2610
47m 33s (- 43m 54s) (130000 52%) 3.2

In [18]:
# save model to path
torch.save(encoder1.state_dict(), MODEL_SAVING_DIR + "210718_encoder1_translation.pkl")
torch.save(attn_decoder1.state_dict(), MODEL_SAVING_DIR + "210718_decoder_translation.pkl")

# Evaluation

In [19]:
def evaluate(encoder, decoder, sentence, LNG, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence, LNG)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]
    
def evaluateRandomly(encoder, decoder, LNG, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0], LNG)
        output_sentence = ''.join(output_words)
        print('<', output_sentence)
        print('')

In [22]:
evaluateRandomly(encoder1, attn_decoder1, "EN")

> True understanding requires the greatest of wisdoms .
= 非上上智，无了了心。
< 对于为的新的意的。生生。<EOS>

> A dog followed me home .
= 一隻狗跟隨我回家。
< 只狗叫我我家。<EOS>

> Flowers bloom .
= 鮮花盛開。
< 海一口叫花。<EOS>

> Tom was shocked .
= 湯姆吃了一驚。
< 汤姆是人了。。<EOS>

> I will try again thank you .
= 我會再試一次, 謝謝您。
< 我希望你再试试。<EOS>

> The storm has died down .
= 暴风雨平息下来了。
< 死了了死了死了了。了。<EOS>

> I don t give a rat s ass .
= 我完全不管。
< 我不有借。有。<EOS>

> He is a comedian .
= 他是喜剧演员。
< 他是个人。<EOS>

> Jack is the tallest boy in his class .
= Jack是班里最高的男孩。
< 他的的的班上上上上的<EOS>

> Tom told Mary that she should stop pretending to be intoxicated .
= 汤姆告诉玛丽她应该停止假装醉酒。
< 汤姆告诉玛丽告诉玛丽会停止假。<EOS>



In [27]:
export_file = codecs.open(RESULT_SAVING_DIR + "ZH_EN_translation.txt", mode="w", encoding="utf8")

for pair in pairs:
    export_file.write("Input: " + str(pair[0]) + "\n")
    export_file.write("Ground Truth: " + str(pair[1]) + "\n")
    output_words, attentions = evaluate(encoder1, attn_decoder1, pair[0], "EN")
    export_file.write("Prediction: " + ''.join(output_words) + "\n\n")